In [7]:
import nltk
nltk.download('gutenberg')

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\HW798MH\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\gutenberg.zip.


True

In [12]:
from nltk.corpus import gutenberg
data = gutenberg.raw('shakespeare-hamlet.txt')

In [16]:
with open('hemlet.txt','w') as file:
    file.write(data)

In [19]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [20]:
with open('hemlet.txt','r') as file:
    text = file.read().lower()

In [22]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])

In [47]:
total_words = len(tokenizer.word_index)+1

In [31]:
input_sequence = []

for line in data.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(len(token_list)):
        n_gram_sequence = token_list[:i+1]  # Corrected slicing of token_list
        input_sequence.append(n_gram_sequence)


In [32]:
input_sequence

[[1],
 [1, 687],
 [1, 687, 4],
 [1, 687, 4, 45],
 [1, 687, 4, 45, 41],
 [1, 687, 4, 45, 41, 1886],
 [1, 687, 4, 45, 41, 1886, 1887],
 [1, 687, 4, 45, 41, 1886, 1887, 1888],
 [1180],
 [1180, 1889],
 [1180, 1889, 1890],
 [1180, 1889, 1890, 1891],
 [57],
 [57, 407],
 [57, 407, 2],
 [57, 407, 2, 1181],
 [57, 407, 2, 1181, 177],
 [57, 407, 2, 1181, 177, 1892],
 [407],
 [407, 1182],
 [407, 1182, 63],
 [408],
 [408, 162],
 [408, 162, 377],
 [408, 162, 377, 21],
 [408, 162, 377, 21, 247],
 [408, 162, 377, 21, 247, 882],
 [18],
 [18, 66],
 [451],
 [451, 224],
 [451, 224, 248],
 [451, 224, 248, 1],
 [451, 224, 248, 1, 30],
 [408],
 [408, 407],
 [451],
 [451, 25],
 [408],
 [408, 6],
 [408, 6, 43],
 [408, 6, 43, 62],
 [408, 6, 43, 62, 1893],
 [408, 6, 43, 62, 1893, 96],
 [408, 6, 43, 62, 1893, 96, 18],
 [408, 6, 43, 62, 1893, 96, 18, 566],
 [451],
 [451, 71],
 [451, 71, 51],
 [451, 71, 51, 1894],
 [451, 71, 51, 1894, 567],
 [451, 71, 51, 1894, 567, 378],
 [451, 71, 51, 1894, 567, 378, 80],
 [451, 

In [35]:
max_sequence = max(len(x) for x in input_sequence)
max_sequence

14

In [50]:
input_sequence = np.array(pad_sequences(input_sequence,maxlen=max_sequence,padding='pre'))
input_sequence

array([[   0,    0,    0, ...,    0,    0,    1],
       [   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]])

In [69]:
X, y = input_sequence[:, :-1], input_sequence[:, -1]

In [70]:
X

array([[   0,    0,    0, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    1],
       [   0,    0,    0, ...,    0,    1,  687],
       ...,
       [   0,    0,    0, ...,  687,    4,   45],
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4]])

In [71]:
y

array([   1,  687,    4, ..., 1047,    4,  193])

In [72]:
import tensorflow as tf

In [73]:
y = tf.keras.utils.to_categorical(y,num_classes=total_words)
y

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [78]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [91]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,LSTM,Embedding

In [94]:
model = Sequential()
model.add(Embedding(input_dim=total_words, output_dim=100, input_length=max_sequence))
model.add(LSTM(150, return_sequences=False))  # Set return_sequences=False
model.add(Dropout(0.2))
#model.add(LSTM(100, return_sequences=False))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [95]:
history = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10)

Epoch 1/10
743/743 ━━━━━━━━━━━━━━━━━━━━ 30s 36ms/step - accuracy: 0.0305 - loss: 6.9373 - val_accuracy: 0.0490 - val_loss: 6.6252
Epoch 2/10
743/743 ━━━━━━━━━━━━━━━━━━━━ 23s 31ms/step - accuracy: 0.0497 - loss: 6.2685 - val_accuracy: 0.0571 - val_loss: 6.6272
Epoch 3/10
743/743 ━━━━━━━━━━━━━━━━━━━━ 27s 36ms/step - accuracy: 0.0596 - loss: 6.0730 - val_accuracy: 0.0599 - val_loss: 6.6759
Epoch 4/10
743/743 ━━━━━━━━━━━━━━━━━━━━ 26s 35ms/step - accuracy: 0.0678 - loss: 5.8585 - val_accuracy: 0.0705 - val_loss: 6.7251
Epoch 5/10
743/743 ━━━━━━━━━━━━━━━━━━━━ 26s 35ms/step - accuracy: 0.0803 - loss: 5.6205 - val_accuracy: 0.0707 - val_loss: 6.8339
Epoch 6/10
743/743 ━━━━━━━━━━━━━━━━━━━━ 22s 29ms/step - accuracy: 0.0904 - loss: 5.4162 - val_accuracy: 0.0717 - val_loss: 6.8976
Epoch 7/10
743/743 ━━━━━━━━━━━━━━━━━━━━ 27s 36ms/step - accuracy: 0.0995 - loss: 5.1936 - val_accuracy: 0.0722 - val_loss: 7.0009
Epoch 8/10
743/743 ━━━━━━━━━━━━━━━━━━━━ 26s 35ms/step - accuracy: 0.1104 - loss: 4.9654 - 